In [2]:
import csv
import time
import openpyxl
import math
import os
os.chdir(r'C:\Users\op1\Desktop\Pyfile\上手風控\data')
class productInfo:
    equityifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    marginifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    def __init__(self, contract=None, month=None,exchange=None,
                 Buy_oi=0,Sell_oi=0,tick=[0,0],margin=0,setP=0,
                 currency=None,TDate=None,Broker='IFP'):
        
        self.contract = contract
        self.month = month
        self.exchange = exchange
        self.Buy_oi = Buy_oi
        self.Sell_oi = Sell_oi
        self.tick = tick
        self.margin = margin
        self.setP = setP
        self.currency = currency
        self.TDate = TDate
        self.Broker = Broker

    def change(self):
        with open('商品轉換.csv','rt',encoding="utf-8") as fphi:
            phifile = csv.reader(fphi)
            check_product = 0
            for i in phifile:
                if self.contract == i[0]:
                    self.contract = i[1]
                    self.exchange = i[2]
                    self.currency = i[3]
                    self.tick[0] = i[4]
                    self.tick[1]= i[5]
                    check_product += 1  
            if check_product == 0 :
                print('請確認',self.contract,'找不到對照商品')
                
    def pimargin(self):
        with open('//10.36.205.170/交易結算部/國外/交易量+未平倉/' +date[5:]+'/保證金.TXT','r') as mar:
            marR = mar.readlines()
            for i in marR:
                if len(i.split())==11 and i.split()[2]==self.contract:
                    self.margin = int(i.split()[6].replace('.00','').replace(',',''))

    def piINFO(self):
        if self.exchange == 'LME':
            return([
                self.Broker ,
                self.contract,
                self.month,
                self.exchange, 
                self.Buy_oi,
                self.Sell_oi,
                #self.tick , 
                self.margin,
                abs((self.Buy_oi + self.Sell_oi) * self.margin),
                self.setP,
                self.currency])
        else:
            return([
                self.Broker ,
                self.contract,
                self.month,
                self.exchange, 
                self.Buy_oi,
                self.Sell_oi,
                #self.tick , 
                self.margin,
                abs((self.Buy_oi - self.Sell_oi) * self.margin),
                self.setP,
                self.currency])

    def proportion(currency):
        usdS = []
        usdS1 = [] # 占比小於1%
        usdN =[]
        
        for i in ifpOI: #([0]contract, [1]month, [2]exchange, [3]oi, [4]tick, [5]margin, [6]setP, [7]currency)
            pi = productInfo(i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7])
#########################BOB################################
            if i[2] == 'LME':
                n = int(i[3][0].replace(',',''))+int(i[3][1].replace(',','')) #LME口數買賣邊相加
                marALL = n *float(i[5].replace(',',''))                     
            else:
                marALL = int(float(i[3].replace(',','').replace('-',''))*float(i[5].replace(',','')))
#########################BOB################################
            marPPT = round(marALL/float(productInfo.equityifp[i[7]].replace(',','')),4)
            if i[7]==currency:
                if marPPT >.01:
                    usdN.append(i[0]+'-'+i[1])
                    usdS.append(marALL)
                else:
                      usdS1.append(marALL)
        usdother = sum(usdS1)
        usdleft =float(productInfo.equityifp[currency].replace(',',''))-float(productInfo.marginifp[currency].replace(',',''))
        usdS.insert(0,usdother)
        usdS.insert(0,usdleft)      
        usdN.insert(0,'other')
        usdN.insert(0,'marginLeft')
#########################BOB################################
        usdS = list(map(lambda x: int(x), usdS))                                #保證金轉整數
        usdN_usdS = list(zip(usdN,usdS))                                        #轉字典
        usdN_usdS = sorted(usdN_usdS, key = lambda x : x[1], reverse = True)    #照保證金大小排序
        usdN_usdS = dict(usdN_usdS)                                             #轉字典方便使用
        return(usdN_usdS)
#########################BOB################################ 

 #--------------------------------------------------上手資料   

fileList = os.listdir()
fileList = [i for i in os.listdir() if i[-3:] == 'txt' and 'R2390' in i]

for i in fileList:
    ifp= open(i, "r")
    R = ifp.readlines()
    ifp.close()
    productInfo.equityifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    productInfo.marginifp = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}

    source_ifp = []
    ifpOI=[]
    IFP = productInfo()

    exchange = ['CBOE','CBOT','CME','COMEX','NYBOT','NYMEX','SGX']



    for z in R:                             #切割資料成list
        k = z.split()
        if k != [] and 'RT' not in k[0]:        #過濾不需要的資料
            source_ifp.append(k)

    IFP.TDate = source_ifp[3][3] + ' ' +  source_ifp[3][2] + ' ' + source_ifp[3][4]
    date0 = time.strptime(IFP.TDate, '%b %d %Y')
    date1 = time.strftime("%Y/%m/%d",date0)   #資料日期
    date = time.strftime("%Y-%m%d",date0)     #sheets名稱資料日期


    for n,i in enumerate(source_ifp):

        if i[0]=='INITIAL'and i[1]=='MARGIN' and len(i) == 7 :  #取保證金總額
            productInfo.marginifp['USD'] = int(i[3][:-5].replace(',',''))

            try:
                productInfo.marginifp['EUR'] = int(i[4][:-5].replace(',',''))
            except ValueError:
                productInfo.marginifp['EUR'] = 0 

            try:
                productInfo.marginifp['SGD'] = int(i[5][:-5].replace(',',''))
            except ValueError:
                productInfo.marginifp['SGD'] = 0 

            try:
                productInfo.marginifp['JPY'] = int(i[6][:-5].replace(',',''))
            except ValueError:
                productInfo.marginifp['JPY'] = 0 


        if i[0]=='TOTAL'and i[1]=='EQUITY' and len(i) == 6 :    #取公司權益數

            productInfo.equityifp['USD'] = int(i[2][:-5].replace(',',''))

            try:
                productInfo.equityifp['EUR'] = int(i[3][:-5].replace(',',''))
            except ValueError:
                productInfo.equityifp['EUR'] = 0
            try:
                productInfo.equityifp['SGD'] = int(i[4][:-5].replace(',',''))
            except ValueError:
                productInfo.equityifp['SGD'] = 0
            try:
                productInfo.equityifp['JPY'] = int(i[5][:-5].replace(',',''))
            except ValueError:
                productInfo.equityifp['JPY'] = 0


        if 'CLOSE' in source_ifp[n]:

            Contract = source_ifp[n-1]      #取合約
            BS = source_ifp[n+1]            #取買賣
                                            #i 取口數、結算價

            if 'E' in Contract:             #有些資料list有'E' 要刪除
                Contract.remove('E')

            if 'LME' in Contract:           #LME商品資料
                IFP.contract = Contract[7]
                IFP.month = Contract[3] + Contract[4] + Contract[5]
                if '*' not in i[1]:         #LME買賣並存 價如其中一邊為空白須自己創1個
                    i.insert(1,'0*')
                IFP.Sell_oi= int('-' + i[0][:-1].replace(',',''))
                IFP.Buy_oi= int(i[1][:-1].replace(',',''))
    ##            IFP.oi = [i[0][:-1],i[1][:-1]]
                IFP.setP = i[4]       
            else:
                if len(Contract) == 10 or len(Contract) == 11 or len(Contract) == 12:
                    IFP.contract = Contract[6]
                    IFP.month = Contract[3] + Contract[4]
                    n = i.index('CLOSE')    #找結算價位置
                    if '/' in i[n+2]:       #商品有分數
                        IFP.setP = i[n+1].replace('.','') + ' ' + i[n+2]
                    else:                   #商品無分數
                        if  Contract[6] == 'MINI':
                            IFP.setP = str(int(float(i[n+1])*100))
                        else:
                            IFP.setP = i[n+1]    
                    if 'LONG:' in BS:
                        IFP.Buy_oi = int(i[0][:-1].replace(',',''))
                    if 'SHORT:' in BS:      #賣單口數加負號
                        IFP.Sell_oi = int('-' + i[0][:-1].replace(',',''))

            IFP.change()
            IFP.pimargin()
    ##        if IFP.piINFO()[2] =='LME':      ######拿掉LME
    ##            continue                
            ifpOI.append(IFP.piINFO())
            IFP.Buy_oi = 0
            IFP.Sell_oi = 0


    curL = ['USD','JPY','EUR']
    for cur in curL:
        usdS = []
        usdN = []
        for i in ifpOI:
            if i[-1] ==cur:
                usdS.append(i[-3])
                usdN.append(i[1]+i[2])

        for i in range(len(ifpOI)):
            if cur in ifpOI[i]:
                marPPS = ifpOI[i][-3]/sum(usdS)
                ifpOI[i].append(marPPS)
                ifpOI[i].append(int(ifpOI[i][-1]*productInfo.marginifp[cur]))

     #--------------------------------------------------填回資料----------------------------------------------- 



    wb = openpyxl.load_workbook('RiskControl.xlsx')

    title=['上手','商品','月份','交易所','買方口數',
           '賣方口數','保證金','加總','結算價','幣別','占比','調整後保證金']

    ifpEqtAndMar = [
                    date1, IFP.Broker,
                    productInfo.equityifp['USD'],productInfo.equityifp['EUR'],
                    productInfo.equityifp['JPY'],productInfo.equityifp['HKD'],
                    productInfo.equityifp['SGD'],
                    productInfo.marginifp['USD'],productInfo.marginifp['EUR'],
                    productInfo.marginifp['JPY'],productInfo.marginifp['HKD'],
                    productInfo.marginifp['SGD'],                
                   ]

    ##....填回每日保證金

    try:                      #判斷sheet是否存在
        ws = wb[date]
    except KeyError:
        wb.create_sheet(date)
        ws = wb[date]
        ws.append(title)

    t = 0                     #判斷資料是否存在
    for i in ws['A']:
        if 'IFP' in i.value:
            t=1
            print(date+ '已有資料')
            break       
    if t == 0 :               #資料填入
        for row in range(len(ifpOI)):
            ws.append(ifpOI[row])

    ##....填回歷史資料

    ws = wb["HistoryData"]

    t = 0
    for i in ws['A']:         #判斷資料是否存在
        if date1 in i.value:
            if 'IFP' in ws.cell(row = i.row , column = 2).value:
                t=1
                break        
    if t == 0 :               #資料填入
        ws.append(ifpEqtAndMar)        

    wb.save('RiskControl.xlsx')
    print(IFP.Broker+date+' OK')



IFP2021-0517 OK
IFP2021-0426 OK
IFP2021-0518 OK
IFP2021-0405 OK
IFP2021-0427 OK
IFP2021-0519 OK
IFP2021-0428 OK
IFP2021-0406 OK
IFP2021-0520 OK
IFP2021-0407 OK
IFP2021-0521 OK
IFP2021-0315 OK
IFP2021-0430 OK
IFP2021-0316 OK
IFP2021-0409 OK
IFP2021-0317 OK
IFP2021-0319 OK
IFP2021-0503 OK
IFP2021-0524 OK
IFP2021-0525 OK
IFP2021-0504 OK
IFP2021-0412 OK
IFP2021-0526 OK
IFP2021-0505 OK
IFP2021-0413 OK
IFP2021-0527 OK
IFP2021-0414 OK
IFP2021-0322 OK
IFP2021-0528 OK
IFP2021-0507 OK
IFP2021-0415 OK


FileNotFoundError: [Errno 2] No such file or directory: '//10.36.205.170/交易結算部/國外/交易量+未平倉/0323/保證金.TXT'